>*you can cross-validate and grid search an entire pipeline*

***preporcessing steps will actomatically occur AFTER each crosss-validation split, which is critical if you want meaingful scores***

In [163]:
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [164]:
df = sns.load_dataset('titanic')

sdf = df[['age', 'sex', 'pclass', 'embark_town', 'survived']]
sdf

,age,sex,pclass,embark_town,survived
0,22.0,male,3,Southampton,0
1,38.0,female,1,Cherbourg,1
2,26.0,female,3,Southampton,1
3,35.0,female,1,Southampton,1
4,35.0,male,3,Southampton,0
...,...,...,...,...,...
886,27.0,male,2,Southampton,0
887,19.0,female,1,Southampton,1
888,NaN,female,3,Southampton,0
889,26.0,male,1,Cherbourg,1


In [165]:
sdf.isna().sum()

age            177
sex              0
pclass           0
embark_town      2
survived         0
dtype: int64

In [166]:
from sklearn.impute import KNNImputer

impute_knn = KNNImputer(n_neighbors=2)
sdf['age'] = impute_knn.fit_transform(sdf[['age']])

In [167]:
sdf.isna().sum()

age            0
sex            0
pclass         0
embark_town    2
survived       0
dtype: int64

In [168]:
sdf = sdf.dropna(axis=False)

In [169]:
sdf.isna().sum()

age            0
sex            0
pclass         0
embark_town    0
survived       0
dtype: int64

In [170]:
X = sdf.drop(columns='survived', axis=1)
X

,age,sex,pclass,embark_town
0,22.000000,male,3,Southampton
1,38.000000,female,1,Cherbourg
2,26.000000,female,3,Southampton
3,35.000000,female,1,Southampton
4,35.000000,male,3,Southampton
...,...,...,...,...
886,27.000000,male,2,Southampton
887,19.000000,female,1,Southampton
888,29.699118,female,3,Southampton
889,26.000000,male,1,Cherbourg


In [171]:
y = sdf['survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: survived, Length: 889, dtype: int64

In [172]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import make_column_transformer

In [173]:
ohe = OneHotEncoder()
vect = CountVectorizer()
ct = make_column_transformer((ohe, ['sex']), (vect, 'embark_town'))

In [174]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='liblinear', random_state=1)

In [175]:
from sklearn.pipeline import make_pipeline
pl = make_pipeline(ct, clf)

***Cross-Validate the entire pipeline (not just the model)***

In [176]:
from sklearn.model_selection import cross_val_score
cross_val_score(pl, X, y, cv=5, scoring='accuracy').mean()

np.float64(0.7862756300387227)

>*preprocessing before splitting the data doesn't properly simulate the reality*
<br>***splitting then preprocessing does simulate the reality.***

---

***Find optimal tuning parameters for the entire pipeline***

In [177]:
#specify parameters values to search (this would be tricky part!)
params = {}
params['columntransformer__countvectorizer__min_df'] = [1,2]
params['logisticregression__C'] = [0.1, 1, 10]
params['logisticregression__penalty'] = ['l11', 'l2']

In [178]:
#try all possible combinations of those parameter values
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pl, params, cv=5, scoring='accuracy')
grid.fit(X, y)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...liblinear'))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'columntransformer__countvectorizer__min_df': [1, 2], 'logisticregression__C': [0.1, 1, ...], 'logisticregression__penalty': ['l11', 'l2']}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time fo

In [179]:
#what was the best score found during the search?
grid.best_score_

np.float64(0.7862756300387227)

In [180]:
#which combination of parameteres produces the best score?
grid.best_params_

{'columntransformer__countvectorizer__min_df': 1,
 'logisticregression__C': 0.1,
 'logisticregression__penalty': 'l2'}